[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/workshops/GEE_Workshop_2024.ipynb)

**Earth Engine Data Converters**

-   Notebook: <https://geemap.org/workshops/GEE_Workshop_2024>
-   Earth Engine: <https://earthengine.google.com>
-   Geemap: <https://geemap.org>

## Introduction

This notebook contains the materials for the workshop **Earth Engine Data Converters** at the 第七届地球空间大数据与云计算前沿会议与集中学习.

### Agenda

This workshop covers the following topics:
- Colab setup
- Vector data conversion
- Raster data extraction
- Exporting Earth Engine data
- Creating timelapse animations


### Prerequisites

-   To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial). To test whether you can use authenticate the Earth Engine Python API, please run [this notebook](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_colab.ipynb) on Google Colab.

## Technical requirements

### Install packages

```bash
conda create -n gee python=3.11
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

In [ ]:
# %pip install geemap pygis mapclassify

### Import libraries

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## Colab setup

Uncomment the following line to get the Earth Engine authorization token. Please treat your token with care and don't share it with anyone. Copy the token to the clipboard.

In [ ]:
# geemap.get_ee_token()

1. Open your Google Colab notebook and click on the `secrets` tab.
2. Create a new secret with the name `EARTHENGINE_TOKEN`.
3. Paste the content from the clipboard into the `Value` input box of the created secret.
4. Toggle the button on the left to allow notebook access to the secret.

![](https://i.imgur.com/Z9R08uU.png)

## Vector data conversion

### From GeoJSON

In [ ]:
in_geojson = "https://github.com/gee-community/geemap/blob/master/examples/data/countries.geojson"
m = geemap.Map()
fc = geemap.geojson_to_ee(in_geojson)
m.add_layer(fc.style(**{"color": "ff0000", "fillColor": "00000000"}), {}, "Countries")
m

### From Shapefile

In [ ]:
url = "https://github.com/gee-community/geemap/blob/master/examples/data/countries.zip"
geemap.download_file(url, overwrite=True)

In [ ]:
in_shp = "countries.shp"
fc = geemap.shp_to_ee(in_shp)

In [ ]:
m = geemap.Map()
m.add_layer(fc, {}, "Countries")
m

### From GeoDataFrame

In [ ]:
import geopandas as gpd

gdf = gpd.read_file(in_shp)
fc = geemap.gdf_to_ee(gdf)

In [ ]:
m = geemap.Map()
m.add_layer(fc, {}, "Countries")
m

### To GeoJSON

In [ ]:
m = geemap.Map()
states = ee.FeatureCollection("TIGER/2018/States")
fc = states.filter(ee.Filter.eq("NAME", "Alaska"))
m.add_layer(fc, {}, "Alaska")
m.center_object(fc, 4)
m

In [ ]:
geemap.ee_to_geojson(fc, filename="Alaska.geojson")

### To Shapefile

In [ ]:
geemap.ee_to_shp(fc, filename="Alaska.shp")

### To GeoDataFrame

In [ ]:
gdf = geemap.ee_to_gdf(fc)
gdf

In [ ]:
gdf.explore()

### To DataFrame

In [ ]:
df = geemap.ee_to_df(fc)
df

### To CSV

In [ ]:
geemap.ee_to_csv(fc, filename="Alaska.csv")

## Raster data extraction

### Extracting values to points

In [ ]:
m = geemap.Map(center=[40, -100], zoom=4)

dem = ee.Image("USGS/SRTMGL1_003")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

m.add_layer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2},
    "Landsat 7",
)
m.add_layer(dem, vis_params, "SRTM DEM", True, 1)
m

In [ ]:
in_shp = "us_cities.shp"
url = "https://github.com/giswqs/data/raw/main/us/us_cities.zip"
geemap.download_file(url)

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
m.add_layer(in_fc, {}, "Cities")

In [ ]:
geemap.extract_values_to_points(in_fc, dem, out_fc="dem.shp")

In [ ]:
geemap.shp_to_gdf("dem.shp")

In [ ]:
geemap.extract_values_to_points(in_fc, landsat7, "landsat.csv")

In [ ]:
geemap.csv_to_df("landsat.csv")

### Extracting pixel values along a transect

In [ ]:
m = geemap.Map(center=[40, -100], zoom=4)
m.add_basemap("TERRAIN")

image = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.add_layer(image, vis_params, "SRTM DEM", True, 0.5)
m

In [ ]:
line = m.user_roi
if line is None:
    line = ee.Geometry.LineString(
        [[-120.2232, 36.3148], [-118.9269, 36.7121], [-117.2022, 36.7562]]
    )
    m.add_layer(line, {}, "ROI")
m.centerObject(line)

In [ ]:
reducer = "mean"
transect = geemap.extract_transect(
    image, line, n_segments=100, reducer=reducer, to_pandas=True
)
transect

In [ ]:
geemap.line_chart(
    data=transect,
    x="distance",
    y="mean",
    markers=True,
    x_label="Distance (m)",
    y_label="Elevation (m)",
    height=400,
)

In [ ]:
transect.to_csv("transect.csv")

## Exporting Earth Engine data

### Exporting images

Add a Landsat image to the map.

In [ ]:
m = geemap.Map()

image = ee.Image("LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318").select(
    ["B5", "B4", "B3"]
)

vis_params = {"min": 0, "max": 0.5, "gamma": [0.95, 1.1, 1]}

m.center_object(image)
m.add_layer(image, vis_params, "Landsat")
m

Add a rectangle to the map.

In [ ]:
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
fc = ee.FeatureCollection(region)
style = {"color": "ffff00ff", "fillColor": "00000000"}
m.add_layer(fc.style(**style), {}, "ROI")

To local drive

In [ ]:
geemap.ee_export_image(image, filename="landsat.tif", scale=30, region=region)

Check image projection.

In [ ]:
projection = image.select(0).projection().getInfo()
projection

In [ ]:
crs = projection["crs"]
crs_transform = projection["transform"]

Specify region, crs, and crs_transform.

In [ ]:
geemap.ee_export_image(
    image,
    filename="landsat_crs.tif",
    crs=crs,
    crs_transform=crs_transform,
    region=region,
)

To Google Drive

In [ ]:
geemap.ee_export_image_to_drive(
    image, description="landsat", folder="export", region=region, scale=30
)

In [ ]:
geemap.download_ee_image(image, "landsat.tif", scale=90)

### Exporting image collections

In [ ]:
point = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(point)
    .filterDate("2008-01-01", "2018-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
collection.aggregate_array("system:index")

To local drive

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=".", scale=10)

To Google Drive

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder="export", scale=10)

### Exporting feature collections

In [ ]:
m = geemap.Map()
states = ee.FeatureCollection("TIGER/2018/States")
fc = states.filter(ee.Filter.eq("NAME", "Alaska"))
m.add_layer(fc, {}, "Alaska")
m.center_object(fc, 4)
m

To local drive

In [ ]:
geemap.ee_to_shp(fc, filename="Alaska.shp")

In [ ]:
geemap.ee_export_vector(fc, filename="Alaska.shp")

In [ ]:
geemap.ee_to_geojson(fc, filename="Alaska.geojson")

In [ ]:
geemap.ee_to_csv(fc, filename="Alaska.csv")

In [ ]:
gdf = geemap.ee_to_gdf(fc)
gdf

In [ ]:
df = geemap.ee_to_df(fc)
df

To Google Drive

In [ ]:
geemap.ee_export_vector_to_drive(
    fc, description="Alaska", fileFormat="SHP", folder="export"
)

## Creating timelapse animations

### Landsat timelapse

In [ ]:
m = geemap.Map(center=[64.838721, -147.763366], zoom=11)
m

Pan and zoom the map to an area of interest. Use the drawing tools to draw a rectangle on the map. If no rectangle is drawn, the default rectangle shown below will be used.

In [ ]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-147.9701, 64.7733, -147.5849, 64.8717)
    m.add_layer(roi)
    m.center_object(roi)

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="Fairbanks.gif",
    start_year=2000,
    end_year=2023,
    start_date="06-01",
    end_date="09-01",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=5,
    title="Landsat Timelapse",
    progress_bar_color="blue",
    mp4=True,
)
geemap.show_image(timelapse)

In [ ]:
m = geemap.Map(center=[64.838721, -147.763366], zoom=11)
m.add_gui("timelapse")
m

In [ ]:
m = geemap.Map()
roi = ee.Geometry.BBox(-115.5541, 35.8044, -113.9035, 36.5581)
m.add_layer(roi)
m.center_object(roi)
m

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="las_vegas.gif",
    start_year=1984,
    end_year=2023,
    bands=["NIR", "Red", "Green"],
    frames_per_second=5,
    title="Las Vegas, NV",
    font_color="blue",
)
geemap.show_image(timelapse)

In [ ]:
m = geemap.Map()
roi = ee.Geometry.BBox(113.8252, 22.1988, 114.0851, 22.3497)
m.add_layer(roi)
m.center_object(roi)
m

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="hong_kong.gif",
    start_year=1990,
    end_year=2022,
    start_date="01-01",
    end_date="12-31",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=3,
    title="Hong Kong",
)
geemap.show_image(timelapse)

### Sentinel-2

In [ ]:
m = geemap.Map(center=[64.838721, -147.763366], zoom=11)
m

Pan and zoom the map to an area of interest. Use the drawing tools to draw a rectangle on the map. If no rectangle is drawn, the default rectangle shown below will be used.

In [ ]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-147.9701, 64.7733, -147.5849, 64.8717)
    m.add_layer(roi)
    m.center_object(roi)

In [ ]:
timelapse = geemap.sentinel2_timelapse(
    roi,
    out_gif="sentinel2.gif",
    start_year=2017,
    end_year=2023,
    start_date="06-01",
    end_date="09-01",
    frequency="year",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=3,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

### MODIS

MODIS vegetation indices

In [ ]:
m = geemap.Map()
m

In [ ]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-18.6983, -36.1630, 52.2293, 38.1446)
    m.add_layer(roi)
    m.center_object(roi)

In [ ]:
timelapse = geemap.modis_ndvi_timelapse(
    roi,
    out_gif="ndvi.gif",
    data="Terra",
    band="NDVI",
    start_date="2000-01-01",
    end_date="2022-12-31",
    frames_per_second=3,
    title="MODIS NDVI Timelapse",
    overlay_data="countries",
)
geemap.show_image(timelapse)

MODIS temperature

In [ ]:
m = geemap.Map()
m

In [ ]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-171.21, -57.13, 177.53, 79.99)
    m.add_layer(roi)
    m.center_object(roi)

In [ ]:
timelapse = geemap.modis_ocean_color_timelapse(
    satellite="Aqua",
    start_date="2018-01-01",
    end_date="2020-12-31",
    roi=roi,
    frequency="month",
    out_gif="temperature.gif",
    overlay_data="continents",
    overlay_color="yellow",
    overlay_opacity=0.5,
)
geemap.show_image(timelapse)

### GOES

In [ ]:
roi = ee.Geometry.BBox(167.1898, -28.5757, 202.6258, -12.4411)
start_date = "2022-01-15T03:00:00"
end_date = "2022-01-15T07:00:00"
data = "GOES-17"
scan = "full_disk"

In [ ]:
timelapse = geemap.goes_timelapse(
    roi, "goes.gif", start_date, end_date, data, scan, framesPerSecond=5
)
geemap.show_image(timelapse)

In [ ]:
roi = ee.Geometry.BBox(-159.5954, 24.5178, -114.2438, 60.4088)
start_date = "2021-10-24T14:00:00"
end_date = "2021-10-25T01:00:00"
data = "GOES-17"
scan = "full_disk"

In [ ]:
timelapse = geemap.goes_timelapse(
    roi, "hurricane.gif", start_date, end_date, data, scan, framesPerSecond=5
)
geemap.show_image(timelapse)

In [ ]:
roi = ee.Geometry.BBox(-121.0034, 36.8488, -117.9052, 39.0490)
start_date = "2020-09-05T15:00:00"
end_date = "2020-09-06T02:00:00"
data = "GOES-17"
scan = "full_disk"

In [ ]:
timelapse = geemap.goes_fire_timelapse(
    roi, "fire.gif", start_date, end_date, data, scan, framesPerSecond=5
)
geemap.show_image(timelapse)